Import relevant packages

In [65]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

Data

In [67]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image=tf.cast(image, tf.float32)
    image/=255.0
    return image,label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

Buffer_size = 10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(Buffer_size)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
training_data=shuffled_train_and_validation_data.skip(num_validation_samples)

Batch_size=100
training_data=training_data.batch(Batch_size)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data))


### MODEL

In [80]:
input_size=784
output_size=10
hidden_layer_size=250
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                             tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [82]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [84]:
Num_epochs=10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(training_data,epochs=Num_epochs,callbacks=[early_stopping],validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/10
540/540 - 5s - 9ms/step - accuracy: 0.9246 - loss: 0.2561 - val_accuracy: 0.9552 - val_loss: 0.1442
Epoch 2/10
540/540 - 4s - 7ms/step - accuracy: 0.9705 - loss: 0.0966 - val_accuracy: 0.9783 - val_loss: 0.0708
Epoch 3/10
540/540 - 4s - 7ms/step - accuracy: 0.9808 - loss: 0.0624 - val_accuracy: 0.9788 - val_loss: 0.0684
Epoch 4/10
540/540 - 4s - 8ms/step - accuracy: 0.9860 - loss: 0.0454 - val_accuracy: 0.9867 - val_loss: 0.0505
Epoch 5/10
540/540 - 3s - 6ms/step - accuracy: 0.9887 - loss: 0.0358 - val_accuracy: 0.9807 - val_loss: 0.0639
Epoch 6/10
540/540 - 4s - 8ms/step - accuracy: 0.9910 - loss: 0.0283 - val_accuracy: 0.9862 - val_loss: 0.0440
Epoch 7/10
540/540 - 4s - 8ms/step - accuracy: 0.9923 - loss: 0.0237 - val_accuracy: 0.9897 - val_loss: 0.0351
Epoch 8/10
540/540 - 4s - 7ms/step - accuracy: 0.9941 - loss: 0.0190 - val_accuracy: 0.9890 - val_loss: 0.0382
Epoch 9/10
540/540 - 3s - 6ms/step - accuracy: 0.9939 - loss: 0.0184 - val_accuracy: 0.9935 - val_loss: 0.0226
E

In [86]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step - accuracy: 0.9821 - loss: 0.0713
Test loss: 0.07. Test accuracy: 98.21%
